In [43]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd

In [44]:
try:
    whole = pd.read_csv("signs.csv")

except:
    print("No dataset found")

In [45]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [46]:
cap = cv2.VideoCapture(0)

ls = []
with mp_hands.Hands(
    model_complexity = 0,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as hands:
    
    while cap.isOpened():

        time_start = time.time()

        retval, frame = cap.read()

        if not retval:
            print("Camera Error; Exiting")
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                
                ls.append([[lmk.x, lmk.y] for lmk in hand_landmarks.landmark])

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        fps = 1 / (time.time() - time_start)

        frame = cv2.putText(frame, f"FPS: {round(fps)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow("Hand Landmarks", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [47]:
# normalize coordinates to be relative centered at landmark 0
ls = np.array(ls)

for l in ls:
    for i in range(21):
        l[i][0] = l[i][0] - l[0][0]
        l[i][1] = l[i][1] - l[0][1]

ls

array([[[0.        , 0.        ],
        [0.45908695, 0.93966192],
        [0.45722294, 0.87219274],
        ...,
        [0.38018355, 0.88256866],
        [0.37242249, 0.85061973],
        [0.36374104, 0.82460916]],

       [[0.        , 0.        ],
        [0.4585081 , 0.92910564],
        [0.45734745, 0.86153013],
        ...,
        [0.3794736 , 0.87008202],
        [0.37016228, 0.84059799],
        [0.36140579, 0.81720763]],

       [[0.        , 0.        ],
        [0.41070643, 0.94497657],
        [0.40154117, 0.89466488],
        ...,
        [0.37833169, 0.80105877],
        [0.37021595, 0.76157999],
        [0.36491904, 0.72927868]],

       ...,

       [[0.        , 0.        ],
        [0.43246037, 0.89584088],
        [0.45068994, 0.84502095],
        ...,
        [0.387622  , 0.76960826],
        [0.38878426, 0.74186671],
        [0.3893435 , 0.7159856 ]],

       [[0.        , 0.        ],
        [0.43251115, 0.89484376],
        [0.45078394, 0.84571469],
        .

In [48]:
reshaped_ls = ls.copy()
reshaped_ls = reshaped_ls.reshape(-1, 42)
reshaped_ls.shape

(503, 42)

In [49]:
df_columns = []
for i in range(21):
    df_columns.append(f"Landmark {i} x")
    df_columns.append(f"Landmark {i} y")

In [50]:
# create dataframe where rows are frames and columns are coordinates of all 21 landmarks (x, y have distinct columns)
df = pd.DataFrame(reshaped_ls, columns=df_columns)
df.drop(["Landmark 0 x", "Landmark 0 y"], axis=1, inplace=True) # can delete landmark 0 since its all just 0 anyway
df.shape

(503, 40)

In [51]:
# add label for handpose (DO NOT USE EMOJIS LOL)
df["Label"] = "a"
df

,Landmark 1 x,Landmark 1 y,Landmark 2 x,Landmark 2 y,Landmark 3 x,Landmark 3 y,Landmark 4 x,Landmark 4 y,Landmark 5 x,Landmark 5 y,...,Landmark 16 y,Landmark 17 x,Landmark 17 y,Landmark 18 x,Landmark 18 y,Landmark 19 x,Landmark 19 y,Landmark 20 x,Landmark 20 y,Label
0,0.459087,0.939662,0.457223,0.872193,0.445482,0.821423,0.434236,0.780295,0.459506,0.869605,...,0.757850,0.391429,0.928150,0.380184,0.882569,0.372422,0.850620,0.363741,0.824609,c
1,0.458508,0.929106,0.457347,0.861530,0.446084,0.809229,0.435359,0.768406,0.460437,0.857658,...,0.756316,0.392689,0.914144,0.379474,0.870082,0.370162,0.840598,0.361406,0.817208,c
2,0.410706,0.944977,0.401541,0.894665,0.388520,0.853488,0.375914,0.821127,0.411019,0.838909,...,0.694906,0.391163,0.848567,0.378332,0.801059,0.370216,0.761580,0.364919,0.729279,c
3,0.418531,0.936610,0.415815,0.887213,0.399207,0.847078,0.380833,0.816062,0.418372,0.828642,...,0.698480,0.367951,0.844490,0.357177,0.798807,0.350121,0.763599,0.344270,0.736226,c
4,0.402729,0.879060,0.405456,0.821639,0.386772,0.783258,0.367376,0.760927,0.396432,0.778981,...,0.655006,0.331824,0.802534,0.328070,0.754426,0.326656,0.721168,0.325087,0.693165,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,0.434439,0.894136,0.451181,0.844983,0.438001,0.807130,0.416522,0.790888,0.438973,0.789610,...,0.677959,0.386087,0.807982,0.388342,0.767840,0.388921,0.740221,0.388836,0.714989,c
499,0.432861,0.895606,0.450341,0.844954,0.436400,0.805916,0.414780,0.787957,0.438667,0.791614,...,0.677607,0.385067,0.808945,0.387964,0.769105,0.389099,0.741275,0.389631,0.715454,c
500,0.432460,0.895841,0.450690,0.845021,0.436729,0.805954,0.414656,0.788805,0.438819,0.790738,...,0.676952,0.384961,0.808946,0.387622,0.769608,0.388784,0.741867,0.389344,0.715986,c
501,0.432511,0.894844,0.450784,0.845715,0.437984,0.806955,0.416417,0.790065,0.438337,0.789139,...,0.676019,0.384983,0.806257,0.387347,0.766988,0.388316,0.739422,0.388797,0.714193,c


In [52]:
try:
    whole = pd.concat([whole, df], axis=0)
except:
    whole = df

whole.shape

(1743, 42)

In [70]:
whole["Label"].value_counts()

okay        664
good        329
not good    247
Name: Label, dtype: int64

In [55]:
# just in case of bad data
def delete_label(df, label):
    return df[df["Label"] != label]

# whole = delete_label(whole, "c")
# whole.shape

(1240, 42)

In [88]:
whole.to_csv("signs_en.csv")